<a href="https://colab.research.google.com/github/mitaligulave/test/blob/main/disaster_project1_mitali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [256]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from gensim.models import Word2Vec

In [257]:

data_train=pd.read_csv('/content/train.csv')
data_test=pd.read_csv('/content/test.csv')

In [258]:
data_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [259]:
data_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [260]:
# Create a binary feature in the train dataset 'has_location' based on whether 'location' is not NaN
data_train['has_location'] = pd.notna(data_train['location']).astype(int)

# Create a binary feature in the test dataset 'has_location' based on whether 'location' is not NaN
data_test['has_location'] = pd.notna(data_test['location']).astype(int)

In [261]:
# drop the column 'location' in the train dataset
data_train.drop(['location'], axis=1, inplace=True)

# drop the column 'location' in the test dataset
data_test.drop(['location'], axis=1, inplace=True)

In [262]:
#  Replace NaN values in the 'keyword' column with a placeholder value, e.g., 'unknown'
data_train['keyword'].fillna('unknown', inplace=True)

 # Initialize the LabelEncoder
label_encoder = LabelEncoder()
data_train['keyword_encoded'] = label_encoder.fit_transform(data_train['keyword']) # Fit and transform the 'keyword' column

# Drop the original 'keyword' column
data_train.drop('keyword', axis=1, inplace=True)

<ipython-input-262-5e910e531c60>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_train['keyword'].fillna('unknown', inplace=True)


In [263]:
# Replace NaN values in the 'keyword' column with a placeholder value, e.g., 'unknown'
data_test['keyword'].fillna('unknown', inplace=True)

 # Initialize the LabelEncoder
label_encoder = LabelEncoder()
data_test['keyword_encoded'] = label_encoder.fit_transform(data_test['keyword']) # Fit and transform the 'keyword' column

# Drop the original 'keyword' column
data_test.drop('keyword', axis=1, inplace=True)

<ipython-input-263-ea05714d6bb9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_test['keyword'].fillna('unknown', inplace=True)


In [264]:
data_train.head()

,id,text,target,has_location,keyword_encoded
0,1,Our Deeds are the Reason of this #earthquake M...,1,0,206
1,4,Forest fire near La Ronge Sask. Canada,1,0,206
2,5,All residents asked to 'shelter in place' are ...,1,0,206
3,6,"13,000 people receive #wildfires evacuation or...",1,0,206
4,7,Just got sent this photo from Ruby #Alaska as ...,1,0,206


In [265]:
data_test.head()

,id,text,has_location,keyword_encoded
0,0,Just happened a terrible car crash,0,206
1,2,"Heard about #earthquake is different cities, s...",0,206
2,3,"there is a forest fire at spot pond, geese are...",0,206
3,9,Apocalypse lighting. #Spokane #wildfires,0,206
4,11,Typhoon Soudelor kills 28 in China and Taiwan,0,206


In [266]:
#Remove html task

import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)



In [267]:
text='#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires'

In [268]:
remove_html_tags(text)

'#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires'

In [269]:
data_train['text']= data_train['text'].apply(remove_html_tags)
data_test['text']= data_test['text'].apply(remove_html_tags)

In [270]:
data_train['text'][6]

'#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas'

In [271]:
data_test['text'][6]

"They'd probably still show more life than Arsenal did yesterday, eh? EH?"

In [272]:
# Remove url
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [273]:
text='On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N'

In [274]:

remove_url(text)

'On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE '

In [275]:
data_train['text']= data_train['text'].apply(remove_url)
data_test['text']= data_test['text'].apply(remove_url)

In [276]:
# Remove punctuation

In [277]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [278]:
exclude=string.punctuation

In [279]:
def remove_punc(text):
    for char in exclude:
        text = text.replace(char,'')
    return text


In [280]:
text='@afterShock_DeLo im speaking from someone that is using a scuf on xb1 most of them people will end up getting on for ps also.'

In [281]:
cleaned_text = remove_punc(text)
print(cleaned_text)

afterShockDeLo im speaking from someone that is using a scuf on xb1 most of them people will end up getting on for ps also


In [282]:
data_train['text']= data_train['text'].apply(remove_punc)
print(data_train)

         id                                               text  target  \
0         1  Our Deeds are the Reason of this earthquake Ma...       1   
1         4              Forest fire near La Ronge Sask Canada       1   
2         5  All residents asked to shelter in place are be...       1   
3         6  13000 people receive wildfires evacuation orde...       1   
4         7  Just got sent this photo from Ruby Alaska as s...       1   
...     ...                                                ...     ...   
7608  10869  Two giant cranes holding a bridge collapse int...       1   
7609  10870  ariaahrary TheTawniest The out of control wild...       1   
7610  10871              M194 0104 UTC5km S of Volcano Hawaii        1   
7611  10872  Police investigating after an ebike collided w...       1   
7612  10873  The Latest More Homes Razed by Northern Califo...       1   

      has_location  keyword_encoded  
0                0              206  
1                0              206

In [283]:
data_test['text']= data_test['text'].apply(remove_punc)
print(data_test)

         id                                               text  has_location  \
0         0                 Just happened a terrible car crash             0   
1         2  Heard about earthquake is different cities sta...             0   
2         3  there is a forest fire at spot pond geese are ...             0   
3         9              Apocalypse lighting Spokane wildfires             0   
4        11      Typhoon Soudelor kills 28 in China and Taiwan             0   
...     ...                                                ...           ...   
3258  10861  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...             0   
3259  10865  Storm in RI worse than last hurricane My citya...             0   
3260  10868                  Green Line derailment in Chicago              0   
3261  10874          MEG issues Hazardous Weather Outlook HWO              0   
3262  10875  CityofCalgary has activated its Municipal Emer...             0   

      keyword_encoded  
0              

In [284]:
# Remove duplicates
data_train = data_train.drop_duplicates()
data_test = data_test.drop_duplicates()

In [285]:
data_train.shape

(7613, 5)

In [286]:
# Remove stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [287]:
def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [288]:
data_train['text']=data_train['text'].apply(remove_stopwords)
print(data_train)

         id                                               text  target  \
0         1  Our Deeds   Reason   earthquake May ALLAH Forg...       1   
1         4              Forest fire near La Ronge Sask Canada       1   
2         5  All residents asked  shelter  place   notified...       1   
3         6  13000 people receive wildfires evacuation orde...       1   
4         7  Just got sent  photo  Ruby Alaska  smoke  wild...       1   
...     ...                                                ...     ...   
7608  10869  Two giant cranes holding  bridge collapse  nea...       1   
7609  10870  ariaahrary TheTawniest The   control wild fire...       1   
7610  10871                 M194 0104 UTC5km S  Volcano Hawaii       1   
7611  10872  Police investigating   ebike collided   car  L...       1   
7612  10873  The Latest More Homes Razed  Northern Californ...       1   

      has_location  keyword_encoded  
0                0              206  
1                0              206

In [289]:
data_test['text']=data_test['text'].apply(remove_stopwords)
print(data_test)

         id                                               text  has_location  \
0         0                  Just happened  terrible car crash             0   
1         2  Heard  earthquake  different cities stay safe ...             0   
2         3     forest fire  spot pond geese  fleeing acros...             0   
3         9              Apocalypse lighting Spokane wildfires             0   
4        11           Typhoon Soudelor kills 28  China  Taiwan             0   
...     ...                                                ...           ...   
3258  10861  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...             0   
3259  10865  Storm  RI worse  last hurricane My cityamp3oth...             0   
3260  10868                     Green Line derailment  Chicago             0   
3261  10874           MEG issues Hazardous Weather Outlook HWO             0   
3262  10875  CityofCalgary  activated  Municipal Emergency ...             0   

      keyword_encoded  
0              

In [290]:
# Remove emoji
import re
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [291]:
data_train['text'] = data_train['text'].apply(remove_emoji)
print(data_train)

         id                                               text  target  \
0         1  Our Deeds   Reason   earthquake May ALLAH Forg...       1   
1         4              Forest fire near La Ronge Sask Canada       1   
2         5  All residents asked  shelter  place   notified...       1   
3         6  13000 people receive wildfires evacuation orde...       1   
4         7  Just got sent  photo  Ruby Alaska  smoke  wild...       1   
...     ...                                                ...     ...   
7608  10869  Two giant cranes holding  bridge collapse  nea...       1   
7609  10870  ariaahrary TheTawniest The   control wild fire...       1   
7610  10871                 M194 0104 UTC5km S  Volcano Hawaii       1   
7611  10872  Police investigating   ebike collided   car  L...       1   
7612  10873  The Latest More Homes Razed  Northern Californ...       1   

      has_location  keyword_encoded  
0                0              206  
1                0              206

In [292]:
data_test['text'] = data_test['text'].apply(remove_emoji)
print(data_test)

         id                                               text  has_location  \
0         0                  Just happened  terrible car crash             0   
1         2  Heard  earthquake  different cities stay safe ...             0   
2         3     forest fire  spot pond geese  fleeing acros...             0   
3         9              Apocalypse lighting Spokane wildfires             0   
4        11           Typhoon Soudelor kills 28  China  Taiwan             0   
...     ...                                                ...           ...   
3258  10861  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...             0   
3259  10865  Storm  RI worse  last hurricane My cityamp3oth...             0   
3260  10868                     Green Line derailment  Chicago             0   
3261  10874           MEG issues Hazardous Weather Outlook HWO             0   
3262  10875  CityofCalgary  activated  Municipal Emergency ...             0   

      keyword_encoded  
0              

In [293]:
# Lowercase all data
data_train['text'] = data_train['text'].str.lower()
print(data_train)

         id                                               text  target  \
0         1  our deeds   reason   earthquake may allah forg...       1   
1         4              forest fire near la ronge sask canada       1   
2         5  all residents asked  shelter  place   notified...       1   
3         6  13000 people receive wildfires evacuation orde...       1   
4         7  just got sent  photo  ruby alaska  smoke  wild...       1   
...     ...                                                ...     ...   
7608  10869  two giant cranes holding  bridge collapse  nea...       1   
7609  10870  ariaahrary thetawniest the   control wild fire...       1   
7610  10871                 m194 0104 utc5km s  volcano hawaii       1   
7611  10872  police investigating   ebike collided   car  l...       1   
7612  10873  the latest more homes razed  northern californ...       1   

      has_location  keyword_encoded  
0                0              206  
1                0              206

In [294]:
data_test['text'] = data_test['text'].str.lower()
print(data_test)

         id                                               text  has_location  \
0         0                  just happened  terrible car crash             0   
1         2  heard  earthquake  different cities stay safe ...             0   
2         3     forest fire  spot pond geese  fleeing acros...             0   
3         9              apocalypse lighting spokane wildfires             0   
4        11           typhoon soudelor kills 28  china  taiwan             0   
...     ...                                                ...           ...   
3258  10861  earthquake safety los angeles ûò safety faste...             0   
3259  10865  storm  ri worse  last hurricane my cityamp3oth...             0   
3260  10868                     green line derailment  chicago             0   
3261  10874           meg issues hazardous weather outlook hwo             0   
3262  10875  cityofcalgary  activated  municipal emergency ...             0   

      keyword_encoded  
0              

In [295]:
# use lemmatizer- to convert to meaningful root word
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [296]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the lemmatized words back into a sentence
    return " ".join(lemmatized_words)


In [297]:
data_train['text'] = data_train['text'].apply(lemmatize_text)
print(data_train)

         id                                               text  target  \
0         1     our deed reason earthquake may allah forgive u       1   
1         4              forest fire near la ronge sask canada       1   
2         5  all resident asked shelter place notified offi...       1   
3         6  13000 people receive wildfire evacuation order...       1   
4         7  just got sent photo ruby alaska smoke wildfire...       1   
...     ...                                                ...     ...   
7608  10869  two giant crane holding bridge collapse nearby...       1   
7609  10870  ariaahrary thetawniest the control wild fire c...       1   
7610  10871                  m194 0104 utc5km s volcano hawaii       1   
7611  10872  police investigating ebike collided car little...       1   
7612  10873  the latest more home razed northern california...       1   

      has_location  keyword_encoded  
0                0              206  
1                0              206

In [298]:
data_test['text'] = data_test['text'].apply(lemmatize_text)
print(data_test)

         id                                               text  has_location  \
0         0                   just happened terrible car crash             0   
1         2  heard earthquake different city stay safe ever...             0   
2         3  forest fire spot pond goose fleeing across str...             0   
3         9               apocalypse lighting spokane wildfire             0   
4        11              typhoon soudelor kill 28 china taiwan             0   
...     ...                                                ...           ...   
3258  10861  earthquake safety los angeles ûò safety faste...             0   
3259  10865  storm ri worse last hurricane my cityamp3other...             0   
3260  10868                      green line derailment chicago             0   
3261  10874            meg issue hazardous weather outlook hwo             0   
3262  10875  cityofcalgary activated municipal emergency pl...             0   

      keyword_encoded  
0              

In [299]:
# Sentence and Word Tokenization:
from nltk.tokenize import word_tokenize, sent_tokenize
data_train['text'] = data_train['text'].fillna('')
def sentence_and_word_tokenize(text):
   # Tokenize into sentences
    sentences = sent_tokenize(text)

    # Tokenize each sentence into words
    word_tokens_in_sentences = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens_in_sentences
data_train['tokenized_text'] = data_train['text'].apply(sentence_and_word_tokenize)
print(data_train)

         id                                               text  target  \
0         1     our deed reason earthquake may allah forgive u       1   
1         4              forest fire near la ronge sask canada       1   
2         5  all resident asked shelter place notified offi...       1   
3         6  13000 people receive wildfire evacuation order...       1   
4         7  just got sent photo ruby alaska smoke wildfire...       1   
...     ...                                                ...     ...   
7608  10869  two giant crane holding bridge collapse nearby...       1   
7609  10870  ariaahrary thetawniest the control wild fire c...       1   
7610  10871                  m194 0104 utc5km s volcano hawaii       1   
7611  10872  police investigating ebike collided car little...       1   
7612  10873  the latest more home razed northern california...       1   

      has_location  keyword_encoded  \
0                0              206   
1                0              2

In [300]:
data_test['tokenized_text'] = data_test['text'].apply(sentence_and_word_tokenize)
print(data_test)

         id                                               text  has_location  \
0         0                   just happened terrible car crash             0   
1         2  heard earthquake different city stay safe ever...             0   
2         3  forest fire spot pond goose fleeing across str...             0   
3         9               apocalypse lighting spokane wildfire             0   
4        11              typhoon soudelor kill 28 china taiwan             0   
...     ...                                                ...           ...   
3258  10861  earthquake safety los angeles ûò safety faste...             0   
3259  10865  storm ri worse last hurricane my cityamp3other...             0   
3260  10868                      green line derailment chicago             0   
3261  10874            meg issue hazardous weather outlook hwo             0   
3262  10875  cityofcalgary activated municipal emergency pl...             0   

      keyword_encoded                  

In [301]:
import spacy.cli
spacy.cli.download("en_core_web_md")
nlp = spacy.load("en_core_web_md")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [302]:
# Load the spaCy model with vectors
nlp = spacy.load("en_core_web_md")

# Function to get word vectors for a list of tokens
def get_word_vectors(tokens):
  if isinstance(tokens, list):  # Check if tokens is a list
        return [nlp(word).vector for word in tokens if isinstance(word, str) and word in nlp.vocab]
  return []  # Return an empty list if the input is not valid

# Function to get the average vector for a list of word vectors
def get_sentence_vector(word_vectors):
    if word_vectors:  # Check if the list is not empty
      return np.mean(np.array(word_vectors), axis=0)
    return np.zeros(nlp.vocab.vectors_length)  # Return a zero vector

# Apply functions to your existing column of tokenized text
data_train['word_vectors'] = data_train['tokenized_text'].apply(get_word_vectors)
data_train['sentence_vector'] = data_train['word_vectors'].apply(get_sentence_vector)

# Display the updated DataFrame
print(data_train[['tokenized_text', 'sentence_vector']])

                                         tokenized_text  \
0     [[our, deed, reason, earthquake, may, allah, f...   
1       [[forest, fire, near, la, ronge, sask, canada]]   
2     [[all, resident, asked, shelter, place, notifi...   
3     [[13000, people, receive, wildfire, evacuation...   
4     [[just, got, sent, photo, ruby, alaska, smoke,...   
...                                                 ...   
7608  [[two, giant, crane, holding, bridge, collapse...   
7609  [[ariaahrary, thetawniest, the, control, wild,...   
7610         [[m194, 0104, utc5km, s, volcano, hawaii]]   
7611  [[police, investigating, ebike, collided, car,...   
7612  [[the, latest, more, home, razed, northern, ca...   

                                        sentence_vector  
0     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4

In [303]:
data_test['word_vectors'] = data_test['tokenized_text'].apply(get_word_vectors)
data_test['sentence_vector'] = data_test['word_vectors'].apply(get_sentence_vector)

# Display the updated DataFrame
print(data_test[['tokenized_text', 'sentence_vector']])

                                         tokenized_text  \
0              [[just, happened, terrible, car, crash]]   
1     [[heard, earthquake, different, city, stay, sa...   
2     [[forest, fire, spot, pond, goose, fleeing, ac...   
3           [[apocalypse, lighting, spokane, wildfire]]   
4        [[typhoon, soudelor, kill, 28, china, taiwan]]   
...                                                 ...   
3258  [[earthquake, safety, los, angeles, ûò, safet...   
3259  [[storm, ri, worse, last, hurricane, my, citya...   
3260               [[green, line, derailment, chicago]]   
3261   [[meg, issue, hazardous, weather, outlook, hwo]]   
3262  [[cityofcalgary, activated, municipal, emergen...   

                                        sentence_vector  
0     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4

In [304]:
# Stack the features together
X_train = np.vstack(data_train['sentence_vector'].values)
X_train1 = np.hstack([X_train, data_train[['keyword_encoded', 'has_location']].values])

X_test = np.vstack(data_test['sentence_vector'].values)
X_test1 = np.hstack([X_test, data_test[['keyword_encoded', 'has_location']].values])

y_train = data_train['target']

In [305]:
X_test1.shape

(3263, 302)

In [306]:
X_train1.shape

(7613, 302)

In [307]:
classifier = RandomForestClassifier(max_depth=10,random_state=42)
classifier.fit(X_train1,y_train)
y_pred=classifier.predict(X_train1)

In [308]:
ac=accuracy_score(y_train, y_pred)
print(ac)

0.7182451070537239


In [309]:
data_sample=pd.read_csv('/content/sample_submission.csv')

In [310]:
data_sample.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [311]:
y_sample=data_sample['target']

In [312]:
print(y_sample)

0       0
1       0
2       0
3       0
4       0
       ..
3258    0
3259    0
3260    0
3261    0
3262    0
Name: target, Length: 3263, dtype: int64


In [313]:
classifier = RandomForestClassifier(max_depth=10,random_state=42)
classifier.fit(X_train1,y_train)
y_pred=classifier.predict(X_test1)

In [314]:
ac=accuracy_score(y_sample, y_pred)
print(ac)

0.7447134538768004
